# Computer Tomography

In this example, we use data from a *computer tomography* (CT) study of a cadaver head: <https://graphics.stanford.edu/data/voldata/>

In [1]:
import glob
import tarfile
import tempfile

import libcarna
import numpy as np
import pooch
import tifffile

Get the data:

In [2]:
data_tar_gz_filepath = pooch.retrieve(
    'https://graphics.stanford.edu/'
    'data/voldata/cthead-16bit.tar.gz',
    known_hash='md5:ea5874800bc1321fecd65ee791ca157b',
)
with tempfile.TemporaryDirectory() as data_dir_path:
    with tarfile.open(data_tar_gz_filepath) as data_tar_gz:
        data_tar_gz.extractall(
            data_dir_path,
            filter=tarfile.fully_trusted_filter,
        )
    data = np.dstack(
        [
            tifffile.imread(filepath) for filepath in
            sorted(glob.glob(f'{data_dir_path}/cthead-*.tif'))
        ]
    )
data.shape, data.dtype

((256, 256, 99), dtype('uint16'))

The data has 99 slices.

## Maximum Intensity Projection

In the code below, we use `normals=True` on the `volume` node; albeit this doesn't make any difference for the *Maximum
Intensity Projection* (MIP), it is benefitial for other, subsequently shown rendering modes.

In [3]:
GEOMETRY_TYPE_VOLUME = 2

# Create and configure frame renderer
mip = libcarna.mip(GEOMETRY_TYPE_VOLUME, sr=400)
r = libcarna.renderer(600, 450, [mip])

# Create and configure scene
root = libcarna.node()
libcarna.volume(
    GEOMETRY_TYPE_VOLUME,
    data / data.max(),
    parent=root,
    spacing=(1, 1, 2),
    local_transform=libcarna.rotate('z', 90).rotate('x', 90),
    normals=True,
)
camera = libcarna.camera(
    parent=root,
    projection=r.frustum(fov=90, z_near=10, z_far=1000),
    local_transform=libcarna.translate(0, 0, 300),
)

# Render
libcarna.imshow(r.render(camera))

The spatial structure of the 3D image is difficult to perceive from that rendering.

For a better visual perception, viewing the data from different angles is benefical, that can be achieved with
as a subtle animation:

In [4]:
# Render as animation
libcarna.imshow(
    libcarna.animate(
        libcarna.animate.rotate_local(camera),
        n_frames=100,
    ).render(r, camera),
)

## Direct Volume Rendering

In a *Direct Volume Rendering* (DVR), surfaces are rendered by simulation of the absorption of light. This simulation
is most realstic, when the spatial orientation of the surfaces can be taken into account, which requires that the
normals of the volume have been computed (this is why we used `normals=True` when we created the `volume` node).

We use a quadratic ramp function to strip out the air from the visualization:

In [5]:
dvr = libcarna.dvr(
    GEOMETRY_TYPE_VOLUME, sr=800, transl=1, diffuse=0.8,
)
dvr.cmap.BrBG(ramp=0.5, rampdegree=2)

libcarna.imshow(
    libcarna.animate(
        libcarna.animate.rotate_local(camera),
        n_frames=100,
    ).render(
        libcarna.renderer(600, 450, [dvr]),
        camera,
    ),
)

## Hounsfield Unit Normalization

The data from CT scanners usually comes in *Hounsfield Units* (HU), that range from -1024 to 3071. In the HU scale, air
roughly corresponds to -1000 HU, water to 0 HU, and bone to +1000 HU. The image intensities in this dataset are not
normalized to the HU scale. Normalization of CT data to the HU scale is beneficial, because it allows to perform
*digital radiograph reconstruciton* (DRR), as shown further below.

To normalize the data to the HU scale ourselves, we need to determine the intensitiy values for air and bone structure,
at least. To this end, we first put two markers for the different structures:

In [6]:
GEOMETRY_TYPE_OPAQUE = 1

# Define marker mesh
ball = libcarna.mesh_factory.create_ball(10)

# Add a marker for bone (red)
libcarna.geometry(
    GEOMETRY_TYPE_OPAQUE,
    parent=root,
    local_transform=libcarna.translate(-43, 20, 0),
    features={
        libcarna.opaque.ROLE_DEFAULT_MESH: ball,
        libcarna.opaque.ROLE_DEFAULT_MATERIAL:
            libcarna.material(color=libcarna.color.RED),
    },
)

# Add a marker for air (green)
libcarna.geometry(
    GEOMETRY_TYPE_OPAQUE,
    parent=root,
    local_transform=libcarna.translate(-80, 0, 0),
    features={
        libcarna.opaque.ROLE_DEFAULT_MESH: ball,
        libcarna.opaque.ROLE_DEFAULT_MATERIAL:
            libcarna.material(color=libcarna.color.GREEN),
    },
)

libcarna.imshow(
    libcarna.animate(
        libcarna.animate.rotate_local(camera),
        n_frames=100,
    ).render(
        libcarna.renderer(600, 450, [
            dvr.replicate(),
            libcarna.opaque(GEOMETRY_TYPE_OPAQUE),
        ]),
        camera,
    ),
)